# Demo SequenceActivity
The sequence activity calls all activities which are specified as sub processes.

In [1]:
import datetime, time
import simpy

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of the subprocesses used in the seqeunce activity
The sequence activity executes three basic activities in a sequence. Since the individual basic activities are executed one after the other, it is necessary to postpone the initialization of the initialization of these activities. This is done using the **postpone_start** parameter.

To ensure that all logging of the basic activities are available in a single log, additional logging is configured to an reporting activity.

In [2]:
reporting_activity = model.BasicActivity(
    env=my_env,
    name="Reporting activity",
    registry=registry,
    duration=0,
)

sub_processes = [
    model.BasicActivity(
        env=my_env,
        name="Basic activity1",
        registry=registry,
        duration=14,
        additional_logs=[reporting_activity],
    ),
    model.BasicActivity(
        env=my_env,
        name="Basic activity2",
        registry=registry,
        duration=5,
        additional_logs=[reporting_activity],
    ),
    model.BasicActivity(
        env=my_env,
        name="Basic activity3",
        registry=registry,
        duration=220,
        additional_logs=[reporting_activity],
    ),
]

## Definition of seqeunce activity

In [3]:
activity = model.SequentialActivity(
    env=my_env,
    name="Sequential process",
    registry=registry,
    sub_processes=sub_processes,
)

In [4]:
model.register_processes([activity])
my_env.run()

In [5]:
plot.vessel_planning([*sub_processes, activity])

In [6]:
display(plot.get_log_dataframe(reporting_activity, [*sub_processes, activity, reporting_activity]))
for act in [*sub_processes, activity]:
    display(plot.get_log_dataframe(act, [*sub_processes, activity, reporting_activity]))

Activity           Timestamp ActivityState
0  Basic activity1 1970-01-01 00:00:00         START
1  Basic activity1 1970-01-01 00:00:14          STOP
2  Basic activity2 1970-01-01 00:00:14         START
3  Basic activity2 1970-01-01 00:00:19          STOP
4  Basic activity3 1970-01-01 00:00:19         START
5  Basic activity3 1970-01-01 00:03:59          STOP

Activity           Timestamp ActivityState
0  Basic activity1 1970-01-01 00:00:00         START
1  Basic activity1 1970-01-01 00:00:14          STOP

Activity           Timestamp ActivityState
0  Basic activity2 1970-01-01 00:00:14         START
1  Basic activity2 1970-01-01 00:00:19          STOP

Activity           Timestamp ActivityState
0  Basic activity3 1970-01-01 00:00:19         START
1  Basic activity3 1970-01-01 00:03:59          STOP

Activity           Timestamp ActivityState        type  \
0  Sequential process 1970-01-01 00:00:00         START         NaN   
1  Sequential process 1970-01-01 00:00:00         START  subprocess   
2  Sequential process 1970-01-01 00:00:14          STOP  subprocess   
3  Sequential process 1970-01-01 00:00:14         START  subprocess   
4  Sequential process 1970-01-01 00:00:19          STOP  subprocess   
5  Sequential process 1970-01-01 00:00:19         START  subprocess   
6  Sequential process 1970-01-01 00:03:59          STOP  subprocess   
7  Sequential process 1970-01-01 00:03:59          STOP         NaN   

                                    ref  
0                                   NaN  
1  dc2210d4-19c3-4ac9-8a6e-f96c2eb005ea  
2  dc2210d4-19c3-4ac9-8a6e-f96c2eb005ea  
3  1847f214-3f57-4d66-a3bf-9c560f3120dd  
4  1847f214-3f57-4d66-a3bf-9c560f3120dd  
5  cfee901f-2cce-4e9d-8b02-0dfe6bf6b13b  
6  cfee901f-2cce-4e9d-8b02-0dfe6bf6b13b  
7                                   NaN